<a href="https://colab.research.google.com/github/rmanzi13/Optimizacion_Compra_Alimentacion/blob/main/recomendacion_llista_compra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
ruta_compras = 'productos_mercado.json'

In [ ]:
with open(ruta_compras,'r') as file:
     data = json.load(file)
# df_productos = pd.DataFrame(data)

In [ ]:
print(data)

In [ ]:
# Elimina la clave "Hoja1"
if "Hoja1" in data:
    del data["Hoja1"]

In [ ]:
# Obtener la lista de productos
productos = data['mercado']

In [ ]:
df_productos = pd.DataFrame(productos)

In [ ]:
# Agregar índices numéricos
df_productos.reset_index(drop=True, inplace=True)

In [ ]:
df_productos.head()

,Producto,Precio,Marca,Categoria
0,Fuet extra Nuestra Alacena bolsa 2 x 170 g,"3,59 €",Nuestra Alacena,Embutidos
1,Chorizo extra dulce Nuestra Alacena bolsa 270 g,"2,40 €",Nuestra Alacena,Embutidos
2,Chorizo extra Nuestra Alacena bandeja 2 x 100 g,"1,46 €",Nuestra Alacena,Embutidos
3,Salchichón extra Nuestra Alacena bandeja 2 x 1...,"1,89 €",Nuestra Alacena,Embutidos
4,Chorizo extra picante Nuestra Alacena bolsa 270 g,"2,40 €",Nuestra Alacena,Embutidos


In [ ]:
df_productos.shape

(198, 4)

In [ ]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Producto   198 non-null    object
 1   Precio     198 non-null    object
 2   Marca      198 non-null    object
 3   Categoria  198 non-null    object
dtypes: object(4)
memory usage: 6.3+ KB


In [ ]:
df_productos['Precio'] = df_productos['Precio'].str.replace(' €', '').str.replace(',', '.').astype(float)

In [ ]:
df_productos.isnull().sum()

Producto     0
Precio       0
Marca        0
Categoria    0
dtype: int64

In [ ]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Producto   198 non-null    object 
 1   Precio     198 non-null    float64
 2   Marca      198 non-null    object 
 3   Categoria  198 non-null    object 
dtypes: float64(1), object(3)
memory usage: 6.3+ KB


In [ ]:
pd.set_option('display.max_rows', None)
unique_compra = df_productos[['Marca','Precio','Categoria']].value_counts()
print(unique_compra)

In [ ]:
pd.set_option('display.max_rows', None)
unique_compra = df_productos['Producto'].value_counts()
print(unique_compra)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def compra_recommender(df_productos):
    try:
        # Limpiar valores nulos o vacíos en las columnas utilizadas
        df_productos = df_productos.dropna(subset=['Precio', 'Marca','Categoria'])

        # Convertir todas las columnas relevantes a cadenas antes de la concatenación
        df_productos['Precio'] = df_productos['Precio'].astype(str)
        df_productos['Marca'] = df_productos['Marca'].astype(str)
        df_productos['Categoria'] = df_productos['Categoria'].astype(str)


        # Concatenar variables para formar la descripción del prodcuto
        df_productos['compra_description'] = (
            df_productos['Precio'].str.lower() + ' ' +
            df_productos['Marca'].str.lower() + ' ' +
            df_productos['Categoria'].str.lower()
        )
        # Utilizamos CountVectorizer para convertir la descripción del producto en una matriz de términos
        vectorizer = CountVectorizer()
        producto_matrix = vectorizer.fit_transform(df_productos['compra_description'])

        # Calculamos la similitud del coseno entre las descripciones de productos
        similarity_matrix = cosine_similarity(producto_matrix, producto_matrix)

        return similarity_matrix

    except Exception as e:
        print(f"Error en compra_recommender: {e}")
        return pd.DataFrame()

# Ejemplo de uso
try:
    # Suponiendo que data_compra es tu DataFrame con los datos de productos
    matriz_similitud_compra = compra_recommender(df_productos)
    print(matriz_similitud_compra)
except Exception as e:
    print(f"Error en el ejemplo de uso: {e}")

[[1.         0.8660254  0.75       ... 0.         0.         0.        ]
 [0.8660254  1.         0.8660254  ... 0.         0.         0.        ]
 [0.75       0.8660254  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.33333333 0.33333333]
 [0.         0.         0.         ... 0.33333333 1.         0.33333333]
 [0.         0.         0.         ... 0.33333333 0.33333333 1.        ]]


In [ ]:
def get_recommendations(matriz_similitud, df_productos, precio, marca, categoria, umbral_similitud=0.5, top_n=5):
    try:
        print("Valores de filtrado:")
        print(f"Precio: {precio}, Marca: {marca}, Categoria: {categoria}")

        # Filtrar el DataFrame según el tipo de producto y el precio
        filtered_df = df_productos[
            (df_productos['Precio'] == str(precio)) &
            (df_productos['Marca'].str.lower() == str(marca).lower()) &
            (df_productos['Categoria'].str.lower() == str(categoria).lower())
        ]

        print("DataFrame filtrado:")
        print(filtered_df)

        # Obtener índices de productos que cumplen con los criterios de filtro
        indices_productos = filtered_df.index.tolist()

        if not indices_productos:
            print("No hay productos que cumplan con los criterios especificados.")
            return []

        # Calcular la puntuación total de recomendación para cada producto
        scores = np.sum(matriz_similitud[indices_productos, :], axis=0)

        # Aplicar umbral de similitud
        relevant_indices = [i for i in range(len(scores)) if scores[i] >= umbral_similitud]

        if not relevant_indices:
            print("No hay productos que superen el umbral de similitud.")
            return []

        # Ordenar los productos según la puntuación de recomendación en orden descendente
        sorted_indices = np.argsort(scores[relevant_indices])[::-1]

        # Obtener los primeros top_n productos recomendados
        top_productos_indices = [relevant_indices[i] for i in sorted_indices[:top_n]]

        # Obtener nombres de productos recomendados
        recommended_products = df_productos.iloc[top_productos_indices]['Producto'].tolist()

        return recommended_products

    except Exception as e:
        print(f"Error en get_recommendations: {e}")
        return []

# Ejemplo de uso
try:
    precio_input = 2.40
    marca_input = 'Nuestra Alacena'
    categoria_input = 'Embutidos'
    umbral_similitud_input = 0.6
    top_n_input = 5

    recomendaciones = get_recommendations(matriz_similitud_compra, df_productos, precio_input, marca_input, categoria_input, umbral_similitud_input, top_n_input)

    if recomendaciones:
        print(f"Productos recomendados para {precio_input} de {marca_input} en la categoría {categoria_input} con umbral de similitud {umbral_similitud_input} y top {top_n_input}:")
        print(recomendaciones)
    else:
        print("No se encontraron recomendaciones.")

except Exception as e:
    print(f"Error en el ejemplo de uso: {e}")


Valores de filtrado:
Precio: 2.4, Marca: Nuestra Alacena, Categoria: Embutidos
DataFrame filtrado:
                                            Producto Precio            Marca  \
1    Chorizo extra dulce Nuestra Alacena bolsa 270 g    2.4  Nuestra Alacena   
4  Chorizo extra picante Nuestra Alacena bolsa 270 g    2.4  Nuestra Alacena   

   Categoria             compra_description  
1  Embutidos  2.4 Nuestra Alacena Embutidos  
4  Embutidos  2.4 Nuestra Alacena Embutidos  
Productos recomendados para 2.4 de Nuestra Alacena en la categoría Embutidos con umbral de similitud 0.6 y top 5:
['Chorizo extra dulce Nuestra Alacena bolsa 270 g', 'Chorizo extra picante Nuestra Alacena bolsa 270 g', 'Fuet extra Nuestra Alacena bolsa 2 x 170 g', 'Chorizo extra Nuestra Alacena bandeja 2 x 100 g', 'Salchichón extra Nuestra Alacena bandeja 2 x 100 g']
